In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv('../data/res_mapping.csv').dropna()
data['res_dt']=pd.to_datetime(data['res_dt'], format="%d.%m.%y %H:%M")
data['dep_soll']=pd.to_datetime(data['dep_soll'], format="%d.%m.%y %H:%M")
data['res_dep_diff']=abs((data['res_dt']-data['dep_soll']).dt.total_seconds()) / 3600 / 24 / 2 # 2 Tage 
data[data['res_dep_diff'] > 1] = 1
data['dep_soll_date']=data['dep_soll'].astype(str).str[:10]
data

,res_id,res_dt,train_nr,reserved,bp_from,bp_to,dep_soll,res_dep_diff,dep_soll_date
0,186372,2021-08-08 21:01:00,507,1,AA,ZUE,2021-08-09 06:31:00,0.197917,2021-08-09
1,1,1,1,1,1,1,1,1.000000,1
2,186374,2021-08-07 17:19:00,508,1,OL,BI,2021-08-09 06:41:00,0.778472,2021-08-09
3,186375,2021-08-09 07:12:00,508,1,BI,YV,2021-08-09 07:16:00,0.001389,2021-08-09
4,186376,2021-08-09 08:06:00,508,1,MOR,GE,2021-08-09 08:18:00,0.004167,2021-08-09
...,...,...,...,...,...,...,...,...,...
54720,246183,2021-10-31 17:35:00,1632,1,BS,BI,2021-10-31 18:03:00,0.009722,2021-10-31
54721,246184,2021-10-31 17:35:00,1633,1,BI,BS,2021-10-31 18:49:00,0.025694,2021-10-31
54722,246185,2021-10-31 18:40:00,1633,1,BI,BS,2021-10-31 18:49:00,0.003125,2021-10-31
54723,246186,2021-10-31 18:51:00,1634,1,LFN,BI,2021-10-31 19:21:00,0.010417,2021-10-31


In [3]:
def weekday_to_number(day):
    if day == 'Mo':
        return 0
    elif day == 'Di':
        return 1
    elif day == 'Mi':
        return 2
    elif day == 'Do':
        return 3
    elif day == 'Fr':
        return 4
    elif day == 'Sa':
        return 5
    elif day == 'So':
        return 6

In [4]:
train_mappings = pd.read_csv('../data/train_mapping.csv').dropna()

In [5]:
for index, row in train_mappings.iterrows():
    vp = row['VP-Tagesleistung (Spaltte R)']
    
    if (len(vp) > 5):
        # Only support one or two day vp
        vp = 'Mo-So'
    days = vp.split('-')
    
    if len(days) == 1:
        train_mappings.at[index, 'VP-Tagesleistung (Spaltte R)'] = [weekday_to_number(days[0])]
    else:
        train_mappings.at[index, 'VP-Tagesleistung (Spaltte R)'] = list(range(weekday_to_number(days[0]), weekday_to_number(days[1])))
train_mappings

KeyboardInterrupt: 

In [ ]:
block_mappings = pd.read_csv('../data/block_mapping.csv').dropna()

In [ ]:
line_stops=pd.read_csv('../data/line_stops.csv')
line_stops = line_stops[np.logical_not(np.logical_and(line_stops['Arrival time'].isnull(), line_stops['Departure time'].isnull()))]
line_stops['Stop time'] = line_stops['Arrival time'].fillna(line_stops['Departure time'])
line_stops['Stop time'] = pd.to_datetime(line_stops['Stop time'], format="%d.%m.%y %H:%M")
line_stops.drop(['Arrival time'], axis=1, inplace=True)
line_stops.drop(['Departure time'], axis=1, inplace=True)
line_stops

In [ ]:
opuic_mapping = pd.read_csv('../data/opuic_mapping.csv')
opuic_mapping

In [ ]:
data['stops'] = None
data.dtypes

In [ ]:
for index, row in data.iterrows():
    try:
        from_station = opuic_mapping[opuic_mapping['Station abbreviation'] == row['bp_from']].iloc[0, 0] #OPUIC
        to_station = opuic_mapping[opuic_mapping['Station abbreviation'] == row['bp_to']].iloc[0, 0]#OPUIC
        # Find all stops and sort by stop time
        stops = line_stops[line_stops['Linie'] == row['train_nr']].sort_values(by='Stop time')

        # Locate the indecies
        from_el = stops[stops['OPUIC'] == from_station]
        to_el = stops[stops['OPUIC'] == to_station]

        if len(from_el) < 1 or len(to_el) < 1:
            # Force stop this row, stop missing
            continue

        data.at[index, 'bp_from'] = float(from_el['OPUIC']) / 9999999
        data.at[index, 'bp_to'] = float(to_el['OPUIC']) / 9999999

        from_el = from_el['Stop time'].iloc[0]
        to_el = to_el['Stop time'].iloc[0]

        stops = stops.loc[(stops['Stop time'] >= from_el) & (stops['Stop time'] <= to_el)]['OPUIC'].tolist()

        if len(stops) < 1:
            print('SOMETHING WENT REALLY WRONG')

        data.at[index, 'stops'] = frozenset([stop for stop in stops])

        weekday = row['dep_soll'].weekday()
        block_nr = train_mappings.loc[
            np.logical_and(
                train_mappings['Zug'] == row['train_nr'], 
                weekday in train_mappings['VP-Tagesleistung (Spaltte R)'])].iloc[0, 3]

        found_block = block_mappings.loc[block_mappings['Block Bezeichnung in Jahresformation Fpl-2022'] == block_nr]

        if len(found_block) > 0:
            data.at[index, 'max_res_count'] = found_block.iloc[0, 1] 
        else:
            data.at[index, 'max_res_count'] = 0
    except:
        pass

In [ ]:
data.dropna(inplace=True)

In [ ]:
avg_weather = pd.read_csv('../data/Average_weather.csv')
avg_weather['validdate'] = avg_weather['validdate'].astype(str).str[:10]
avg_weather

In [ ]:
for index, row in data.iterrows():
    num_of_res = sum(data.loc[np.logical_and(np.logical_and(
        data['train_nr'] == row['train_nr'],
        data['dep_soll_date'] == row['dep_soll_date']),
        len(data['stops'].isin(row['stops'])) > 0)]['reserved'])
    
    max_res = row['max_res_count']
    if max_res <= 0:
        percent_of_res = 1 
    else:
        percent_of_res = 1 / max_res * num_of_res
    
    if percent_of_res < 0:
        percent_of_res = 0
    if percent_of_res > 1:
        percent_of_res = 1
    
    data.at[index, 'percent_of_res'] = percent_of_res
    
    data.at[index, 'leisure_biking:idx'] = avg_weather[avg_weather['validdate'] == row['dep_soll_date']]['leisure_biking:idx'].values[0]

In [ ]:
data.to_csv('../data/output.csv', columns = ['bp_from', 'bp_to', 'percent_of_res', 'leisure_biking:idx', 'res_dep_diff'], index=False)